### Import Libraries

In [1]:
# Used to change filepaths
from pathlib import Path

# We set up matplotlib, pandas, and the display function
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import os
from tqdm.notebook import tqdm

In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data_dir = "/kaggle/input/dsgt-dog-breed-identification/"
train_dir = data_dir + '/train/'
test_dir = data_dir + 'test/'

In [4]:
labels_df = pd.read_csv(data_dir + 'labels.csv')
sample_df = pd.read_csv(data_dir + 'sample_submission.csv')
labels_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [5]:
sample_df.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,001a5f3114548acdefa3d4da05474c2e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


In [6]:
dog_breeds = sorted(labels_df['breed'].unique())
n_breeds = len(dog_breeds)
print(n_breeds)

120


In [7]:
breed_to_num = dict(zip(dog_breeds, range(n_breeds)))

In [8]:
def img_to_array_train(data_dir, labels_df, img_size = (224,224,3)):

    img_ids = labels_df['id']
    img_labels = labels_df['breed']
    count = len(img_ids)

    X = np.zeros([count, *img_size], dtype=np.uint8)
    y = np.zeros([count, 1], dtype=np.uint8)

    for i in range(count):
        img_id = img_ids[i]
        img_dir = os.path.join(data_dir, f"{img_id}.jpg")
        img_array = load_img(img_dir, target_size = img_size)
        X[i] = img_array
        
        img_breed = img_labels[i]
        y[i] = breed_to_num[img_breed]
    
    y = to_categorical(y)
    
    return X, y

# double check size
img_size = (224,224,3)
X, y = img_to_array_train(train_dir, labels_df, img_size)
print('Train Data Size: ', X.shape)
print('Label Size: ', y.shape)

Train Data Size:  (10222, 224, 224, 3)
Label Size:  (10222, 120)


In [9]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
input_layer = Input(img_size)
preprocessor = Lambda(preprocess_input)(input_layer)
base_model = InceptionV3(weights = 'imagenet', include_top = False, input_shape = img_size)(preprocessor)
avg = GlobalAveragePooling2D()(base_model)
features_model = Model(inputs = input_layer, outputs = avg)
train_features = features_model.predict(X, batch_size = 64, verbose=1)

160/160 [==============================] - 18s 70ms/step


In [10]:
from keras.callbacks import EarlyStopping
EarlyStop_callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)
my_callback = [EarlyStop_callback]

In [11]:
finetune_model = keras.models.Sequential([InputLayer(train_features.shape[1:]),
                                       Dropout(0.7),
                                       Dense(n_breeds, activation='softmax')])
finetune_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
finetune_model.fit(train_features, y, batch_size = 128, epochs = 60, validation_split = 0.1, callbacks = my_callback)

Epoch 1/60
72/72 [==============================] - 2s 6ms/step - loss: 2.8677 - accuracy: 0.3834 - val_loss: 0.6994 - val_accuracy: 0.7898
Epoch 2/60
72/72 [==============================] - 0s 3ms/step - loss: 1.0824 - accuracy: 0.7016 - val_loss: 0.5832 - val_accuracy: 0.8260
Epoch 3/60
72/72 [==============================] - 0s 3ms/step - loss: 0.8628 - accuracy: 0.7562 - val_loss: 0.5748 - val_accuracy: 0.8289
Epoch 4/60
72/72 [==============================] - 0s 4ms/step - loss: 0.7940 - accuracy: 0.7764 - val_loss: 0.5834 - val_accuracy: 0.8192
Epoch 5/60
72/72 [==============================] - 0s 4ms/step - loss: 0.7023 - accuracy: 0.7951 - val_loss: 0.6052 - val_accuracy: 0.8280
Epoch 6/60
72/72 [==============================] - 0s 3ms/step - loss: 0.6494 - accuracy: 0.8080 - val_loss: 0.6060 - val_accuracy: 0.8309
Epoch 7/60
72/72 [==============================] - 0s 3ms/step - loss: 0.5829 - accuracy: 0.8185 - val_loss: 0.6011 - val_accuracy: 0.8280
Epoch 8/60
72/72 [==

In [12]:
def img_to_array_test(data_dir, labels_dataframe, img_size = (224,224,3)):
    img_ids = labels_dataframe['id']
    count = len(img_ids)
    
    X = np.zeros([count, img_size[0], img_size[1], 3], dtype=np.uint8)
    
    for i in range(count):
        img_id = img_ids[i]
        img_dir = os.path.join(data_dir, f"{img_id}.jpg")
        img_array = load_img(img_dir, target_size=img_size)
        X[i] = img_array
        
    return X

test_set = img_to_array_test(test_dir, sample_df, img_size)
print('Test Data Size: ', test_set.shape)

Test Data Size:  (10357, 224, 224, 3)


In [13]:
test_features = features_model.predict(test_set, batch_size = 64, verbose = 1)

162/162 [==============================] - 11s 71ms/step


In [14]:
predictions = finetune_model.predict(test_features, batch_size=128)

81/81 [==============================] - 0s 2ms/step


In [15]:
for b in dog_breeds:
    sample_df[b] = predictions[:, breed_to_num[b]]
    
sample_df.to_csv('pred.csv', index=None)

In [16]:
sample_df.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.000015,0.000026,0.000012,5.039916e-06,0.000005,6.295465e-07,6.195021e-07,0.000007,0.000012,...,0.000003,1.173998e-05,0.000020,0.000004,0.000004,1.479614e-05,5.483101e-07,0.000012,7.783317e-06,0.000012
1,00102ee9d8eb90812350685311fe5890,0.000001,0.000001,0.000006,8.657503e-07,0.000003,1.456463e-06,1.391407e-06,0.000002,0.000002,...,0.000003,8.761674e-08,0.000007,0.000003,0.000002,2.605378e-07,1.747578e-05,0.000001,3.429163e-07,0.000002
2,0012a730dfa437f5f3613fb75efcd4ce,0.000002,0.297687,0.000128,1.716373e-05,0.000002,6.614669e-07,1.661628e-06,0.000004,0.000019,...,0.000004,2.863637e-05,0.000022,0.000079,0.000072,6.118159e-05,6.254219e-06,0.005621,1.432452e-04,0.000012
3,001510bc8570bbeee98c8d80c8a95ec1,0.005424,0.000282,0.000028,1.601197e-06,0.000043,8.242835e-07,6.876427e-06,0.000037,0.000040,...,0.000001,4.577805e-06,0.000020,0.000206,0.000006,1.075154e-07,2.450480e-07,0.000196,8.684729e-08,0.000016
4,001a5f3114548acdefa3d4da05474c2e,0.010577,0.000183,0.000021,1.988567e-06,0.000005,1.612073e-06,1.506424e-05,0.000008,0.000196,...,0.000027,1.018537e-06,0.000118,0.000038,0.000021,1.245101e-06,1.305818e-05,0.000027,1.024395e-06,0.000122
